In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Fix the import path
from model.model import run

# Run model with no_ai=True (note the correct keyword argument syntax)
results = run(no_ai=True)

# Convert results to a DataFrame for plotting
xdf = pd.DataFrame({
    't': list(range(len(results['sheep_history']))),
    's': results['sheep_history'],
    'w': results['wolf_history']
})

# Create the plot
sns.lineplot(data=pd.melt(xdf, id_vars=['t']), x='t', y='value', hue='variable', palette=['cadetblue', 'darkred'])
plt.legend(frameon=False, ncol=2, loc='lower center', bbox_to_anchor=(.5, -.2))
sns.despine(left=True, bottom=True)
plt.subplots_adjust(bottom=0.2)
plt.title('Wolf-Sheep Population Dynamics (No AI)')